# Predicción y Cálculo Mostradores

**Trabajo Fin de Máster**

**Sistema de predicción para prevenir la congestión en mostradores de facturación en aeropuertos**

Autor: David Cortés Alvarez

Máster en Inteligencia Artificial

## Librerías

In [1]:
# Tratamiento de Datos
# -----------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import qgrid
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

# Gráficos
# -----------------------------------------------------------------------------------


# Modelos
# -----------------------------------------------------------------------------------


# Varios
# -----------------------------------------------------------------------------------
import math
import pickle

## Funciones definidas 

In [2]:
def dataPre(dataset, aeropuertos,debug=False):
    # Eliminamos aquellos registros que no son de los aeropuertos a tratar
    dataset = dataset[dataset.APD.isin(aeropuertos)]
    if debug: print("Tarea 01: Registros de vuelos filtrados por aeropuertos --> OK")
    
    # Eliminamos los que no tienen oferta u ocupación
    dataset = dataset[(dataset['OFERTA'] > 0) & (dataset['OCUPACION'] > 0)]
    if debug: print("Tarea 02: Eliminar registros sin Oferta u Ocupación --> OK")
    
    # Convertimos el identificador de vuelo a número
    dataset['VUELO'] = dataset['VUELO'].astype('str')
    dataset['VUELO'] = dataset['VUELO'].str.replace(r'^(0+)(?!$)', '', regex=True).fillna('0')
    dataset['VUELO'] = dataset['VUELO'].astype(int)
    if debug: print("Tarea 03: Conversión del número de vuelo --> OK")
    
    # Creamos una columna nueva de los minutos del día que se produce la salida del vuelo. 
    dataset.insert(len(dataset.columns), 'SALIDA_MINUTOS_DIA', 
                   dataset.HORA.astype(str).apply(lambda x: x.split(':'))
                   .apply(lambda x: int(x[0]) * 60 + int(x[1])))
    if debug: print("Tarea 04: Creación de la columna SALIDA_MINUTOS_DIA --> OK")

    # OUTLIERS: Filtramos los registros incongruentes
    dataset = dataset[dataset['OFERTA'] >= dataset['PAX_V']]
    dataset = dataset[dataset['OFERTA'] >= dataset['PAX']]
    dataset = dataset[dataset['OFERTA'] <= dataset['CMAX']]
    if debug: print("Tarea 05: Tratamiento Outliers --> OK")
    
    # NORMALIZACIÓN y ESCALADO
    # Insertamos una nueva columna FACTOR_OFERTA basado en OFERTA a un valor ponderable entre 0-1 
    #con respecto a CCMAX 
    dataset.insert(len(dataset.columns), 'FACTOR_OFERTA', ((dataset['OFERTA'] * 100) / dataset['CMAX'])/100)
    # Transformamos OCUPACION por FACTOR_OCUPACION
    dataset.insert(len(dataset.columns), 'FACTOR_OCUPACION', (dataset['OCUPACION'] / 100))
    if debug: print("Tarea 06: Normalización y Escalado --> OK")
    
    # TRATAMIENTO DE NULOS
    #Borramos aquellos registros que contienen nulos en PAX, PAX_M y PAX_V
    dataset = dataset.dropna(subset=['PAX']).dropna(subset=['PAX_V'])
    if debug: print("Tarea 07: Tratamiento de nulos --> OK")
    
    return dataset

## Carga de Datos

In [3]:
# Parámetros de entrada
Escalas_Seleccionadas = ['LPA', 'ACE', 'FUE', 'TFN', 'TFS', 'SPC', 'VDE','GMZ']
Fecha = '01/11/2021'

# Ruta de dataset a cargar
FileNamePrediction = "TFM_FileIN_Dataset_Prediction.csv"
FileName_ModeloPre_A120 = "TFM_FileOUT_ModeloXGB_A120.pkl"
FileName_ModeloPre_A60 = "TFM_FileOUT_ModeloXGB_A60.pkl"
FileName_ModeloPre_A0 = "TFM_FileOUT_ModeloXGB_A0.pkl"
FileNameOUT="TFM_FileOUT_"

Fecha = datetime.strptime(Fecha, '%d/%m/%Y')
day = Fecha.day
month = Fecha.month
year =Fecha.year

In [4]:
data = pd.read_csv(FileNamePrediction, header=0, sep=';')

In [5]:
# Filtramos los vuelos que no son del día seleccionado
dataFilter = data[(data["ANIO"] == year) & (data["MES"] == month) & (data["DIA"] == day)]

In [6]:
# Visualizamos ahora los datos del Dataset
qgrid_widget = qgrid.show_grid(dataFilter, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
# Pre-Procesado de datos
dataAdjusted = dataPre(dataFilter,Escalas_Seleccionadas,debug=True)

Tarea 01: Registros de vuelos filtrados por aeropuertos --> OK
Tarea 02: Eliminar registros sin Oferta u Ocupación --> OK
Tarea 03: Conversión del número de vuelo --> OK
Tarea 04: Creación de la columna SALIDA_MINUTOS_DIA --> OK
Tarea 05: Tratamiento Outliers --> OK
Tarea 06: Normalización y Escalado --> OK
Tarea 07: Tratamiento de nulos --> OK


In [8]:
dataAdjusted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 1 to 194
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_VUELO            173 non-null    object 
 1   APD                 173 non-null    object 
 2   APA                 173 non-null    object 
 3   SEGMENTO            173 non-null    object 
 4   CIA                 173 non-null    object 
 5   VUELO               173 non-null    int64  
 6   FLOTA               173 non-null    object 
 7   PROPIETARIO_VUELO   173 non-null    object 
 8   FECHA_SALIDA_PROG   173 non-null    object 
 9   ANIO                173 non-null    int64  
 10  MES                 173 non-null    int64  
 11  DIA                 173 non-null    int64  
 12  DIA_SEMANA          173 non-null    int64  
 13  HORA                173 non-null    object 
 14  FESTIVO             173 non-null    int64  
 15  MATR                173 non-null    object 
 16  FRANJA_H

## Predicción de Pax presentados en mostrador

In [9]:
col_res = ['ID_VUELO','DIA','ANIO','CIA','FECHA_SALIDA_PROG','PAX_V','HORA','OFERTA','OCUPACION']

In [10]:
# Cargamos los modelos de predicción
with open(FileName_ModeloPre_A120, 'rb') as file:  
    ModeloPre_A120 = pickle.load(file)

with open(FileName_ModeloPre_A60, 'rb') as file:  
    ModeloPre_A60 = pickle.load(file)
    
with open(FileName_ModeloPre_A0, 'rb') as file:  
    ModeloPre_A0 = pickle.load(file)
    
print(ModeloPre_A120)
print(ModeloPre_A60)
print(ModeloPre_A0)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=None,
             subsample=0.9, tree_method='exact', validate_parameters=1,
             verbosity=1)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
         

In [11]:
# Convertir los valores categóricos en numéricos
dataPredict = pd.get_dummies(dataAdjusted, columns = ['APD','APA','SEGMENTO', 'FLOTA',
                                                     'PROPIETARIO_VUELO','FRANJA_HORARIA','OPERADOR',
                                                     'SECTOR','MATR'])

### Predicción A120

In [12]:
dataPredictA120 = dataPredict.reindex(columns=ModeloPre_A120.feature_names).fillna(0).values

In [13]:
dataPredict_Result_A120 = ModeloPre_A120.predict(dataPredictA120)

### Predicción A60

In [14]:
dataPredictA60 = dataPredict.reindex(columns=ModeloPre_A60.feature_names).fillna(0).values

In [15]:
dataPredict_Result_A60 = ModeloPre_A60.predict(dataPredictA60)

### Predicción A0

In [16]:
dataPredictA0 = dataPredict.reindex(columns=ModeloPre_A0.feature_names).fillna(0).values

In [17]:
dataPredict_Result_A0 = ModeloPre_A0.predict(dataPredictA0)

In [18]:
# Añadimos a los datos de entrada
dataAdjusted['A120'] = pd.DataFrame(dataPredict_Result_A120, columns = ['A120']).values
dataAdjusted['A60'] = pd.DataFrame(dataPredict_Result_A60, columns = ['A60']).values
dataAdjusted['A0'] = pd.DataFrame(dataPredict_Result_A0, columns = ['A0']).values

In [19]:
# Ajustamos los valores de las columnas de predicción:
#   - Si son negativos, dejarlos a 0
dataAdjusted['A120'] = dataAdjusted['A120'].apply(lambda x : x if x > 0 else 0)
dataAdjusted['A60'] = dataAdjusted['A60'].apply(lambda x : x if x > 0 else 0)
dataAdjusted['A0'] = dataAdjusted['A0'].apply(lambda x : x if x > 0 else 0)

In [20]:
#   - Redondear al alza
dataAdjusted['A120'] = dataAdjusted['A120'].apply(np.ceil)
dataAdjusted['A60'] = dataAdjusted['A60'].apply(np.ceil)
dataAdjusted['A0'] = dataAdjusted['A0'].apply(np.ceil)

In [21]:
# Convertimos las columnas a INT
dataAdjusted = dataAdjusted.astype({"A120": int, "A60": int, "A0": int})

In [22]:
dataAdjusted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 1 to 194
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_VUELO            173 non-null    object 
 1   APD                 173 non-null    object 
 2   APA                 173 non-null    object 
 3   SEGMENTO            173 non-null    object 
 4   CIA                 173 non-null    object 
 5   VUELO               173 non-null    int64  
 6   FLOTA               173 non-null    object 
 7   PROPIETARIO_VUELO   173 non-null    object 
 8   FECHA_SALIDA_PROG   173 non-null    object 
 9   ANIO                173 non-null    int64  
 10  MES                 173 non-null    int64  
 11  DIA                 173 non-null    int64  
 12  DIA_SEMANA          173 non-null    int64  
 13  HORA                173 non-null    object 
 14  FESTIVO             173 non-null    int64  
 15  MATR                173 non-null    object 
 16  FRANJA_H

In [23]:
# Exportamos a Excel
file = FileNameOUT+'Predicciones.xlsx'
dataAdjusted.to_excel (file, index = False, header=True)
print("Fichero de resultado creado: " + file)

Fichero de resultado creado: TFM_FileOUT_Predicciones.xlsx


## Algoritmo de agrupación de pasajeros por intervalos de tiempo por día

In [24]:
# Creamos las variables de seguimiento
dfAeropuertos = pd.DataFrame()
minIntervalo = 60          # Duración de cada intervalo (minutos)

# Recorremos la lista de Aeropuertos
for aeropuerto in Escalas_Seleccionadas:
    print("Escala: " + aeropuerto)
    dayInterval = np.zeros(24) # Vector de Intervalos. Ejemplo: Pos 0 --> 00:00h a 01:00h
    InicioIntervalo = 0
    FinIntervalo = InicioIntervalo + minIntervalo

    # Recorremos los intervalos del día
    for pos in range(0,len(dayInterval)):
        print("Intervalo: " + str(pos) + ", Entre las " + 
              str(int(InicioIntervalo/minIntervalo)) + ":00h y las " +
              str(int(FinIntervalo/minIntervalo)) + ":00h")
        TotalPaxIntervalo = 0

        # Recorremos los vuelos
        for index, vuelo in dataAdjusted.iterrows():
            if vuelo.APD == aeropuerto:
                InicioBloque = -1
                # Calculamos los inicios de cada intervalo del vuelo
                InicioA120 = vuelo.SALIDA_MINUTOS_DIA - (minIntervalo*3)
                InicioA60 = vuelo.SALIDA_MINUTOS_DIA - (minIntervalo*2)
                InicioA0 = vuelo.SALIDA_MINUTOS_DIA - (minIntervalo*1)

                if InicioIntervalo <= InicioA120 < FinIntervalo:
                    InicioBloque = InicioA120
                    NumPaxIntervalo = vuelo.A120
                else:
                    if InicioIntervalo <= InicioA60 < FinIntervalo:
                        InicioBloque = InicioA60
                        NumPaxIntervalo = vuelo.A60 
                    else:
                        if InicioIntervalo <= InicioA0 < FinIntervalo:
                            InicioBloque = InicioA0
                            NumPaxIntervalo = vuelo.A0

                if InicioBloque != -1:
                    PaxPonderado = math.ceil(((FinIntervalo-InicioBloque)*NumPaxIntervalo)/minIntervalo)
                    TotalPaxIntervalo = TotalPaxIntervalo + PaxPonderado

        dayInterval[pos] = int(TotalPaxIntervalo)
        print("     >Pax = ", dayInterval[pos])

        # Actualizamos los minutos de InicioIntervalo y FinIntervalo
        InicioIntervalo = FinIntervalo
        FinIntervalo = FinIntervalo + minIntervalo

    dfAeropuertos[aeropuerto] = dayInterval.tolist()

print(dfAeropuertos)

Escala: LPA
Intervalo: 0, Entre las 0:00h y las 1:00h
     >Pax =  0.0
Intervalo: 1, Entre las 1:00h y las 2:00h
     >Pax =  0.0
Intervalo: 2, Entre las 2:00h y las 3:00h
     >Pax =  0.0
Intervalo: 3, Entre las 3:00h y las 4:00h
     >Pax =  0.0
Intervalo: 4, Entre las 4:00h y las 5:00h
     >Pax =  6.0
Intervalo: 5, Entre las 5:00h y las 6:00h
     >Pax =  31.0
Intervalo: 6, Entre las 6:00h y las 7:00h
     >Pax =  53.0
Intervalo: 7, Entre las 7:00h y las 8:00h
     >Pax =  49.0
Intervalo: 8, Entre las 8:00h y las 9:00h
     >Pax =  63.0
Intervalo: 9, Entre las 9:00h y las 10:00h
     >Pax =  62.0
Intervalo: 10, Entre las 10:00h y las 11:00h
     >Pax =  40.0
Intervalo: 11, Entre las 11:00h y las 12:00h
     >Pax =  61.0
Intervalo: 12, Entre las 12:00h y las 13:00h
     >Pax =  72.0
Intervalo: 13, Entre las 13:00h y las 14:00h
     >Pax =  84.0
Intervalo: 14, Entre las 14:00h y las 15:00h
     >Pax =  79.0
Intervalo: 15, Entre las 15:00h y las 16:00h
     >Pax =  86.0
Intervalo: 16,

     >Pax =  0.0
Intervalo: 19, Entre las 19:00h y las 20:00h
     >Pax =  0.0
Intervalo: 20, Entre las 20:00h y las 21:00h
     >Pax =  0.0
Intervalo: 21, Entre las 21:00h y las 22:00h
     >Pax =  0.0
Intervalo: 22, Entre las 22:00h y las 23:00h
     >Pax =  0.0
Intervalo: 23, Entre las 23:00h y las 24:00h
     >Pax =  0.0
Escala: VDE
Intervalo: 0, Entre las 0:00h y las 1:00h
     >Pax =  0.0
Intervalo: 1, Entre las 1:00h y las 2:00h
     >Pax =  0.0
Intervalo: 2, Entre las 2:00h y las 3:00h
     >Pax =  0.0
Intervalo: 3, Entre las 3:00h y las 4:00h
     >Pax =  0.0
Intervalo: 4, Entre las 4:00h y las 5:00h
     >Pax =  0.0
Intervalo: 5, Entre las 5:00h y las 6:00h
     >Pax =  1.0
Intervalo: 6, Entre las 6:00h y las 7:00h
     >Pax =  1.0
Intervalo: 7, Entre las 7:00h y las 8:00h
     >Pax =  3.0
Intervalo: 8, Entre las 8:00h y las 9:00h
     >Pax =  1.0
Intervalo: 9, Entre las 9:00h y las 10:00h
     >Pax =  13.0
Intervalo: 10, Entre las 10:00h y las 11:00h
     >Pax =  14.0
Interv

# Teoría de Colas

In [25]:
def sumatoriaRo(c,ro):
    result=0
    for n in range(0,c):
        result = result + ((ro**n)/math.factorial(n))
    return result

In [26]:
tMEspera = 5
tMServicio = 1.08
costeMostrador = 100
conversionMedida = 60 #Minutos en 1h

NumPax = 95

qtLambda = NumPax/60
qtMu = tMServicio
qtC = 2
qtRo = (qtLambda/qtMu)
qtRoSat = (qtLambda/(qtC*qtMu))
qtP0 = (1/(sumatoriaRo(qtC,qtRo)+(((qtRo**qtC)/math.factorial(qtC))*(1/(1-qtRoSat)))))
qtLq = (qtP0*((qtRo**(qtC+1))/(math.factorial(qtC-1)*((qtC-qtRo)**2))))
qtLs =((qtLambda*qtMu*(qtRo**qtC)*qtP0)/(math.factorial(qtC-1)*((qtC*qtMu)-qtLambda)**2))+qtRo
qtWq = qtLq/qtLambda
qtWs = qtLs/qtLambda

print("Ro = "+str(qtRo))
print("RoSat = "+str(qtRoSat))
print("P0 = "+str(qtP0))
print("Lq = "+str(qtLq))
print("Ls = "+str(qtLs))
print("Wq = "+str(qtWq))
print("Ws = "+str(qtWs))

Ro = 1.4660493827160492
RoSat = 0.7330246913580246
P0 = 0.15405164737310778
Lq = 1.7025895334817887
Ls = 3.168638916197838
Wq = 1.0753197053569192
Ws = 2.001245631282845


In [27]:
# Recorremos el resultado de los intervalos de aeropuertos
for aeropuerto in Escalas_Seleccionadas: 
    print('Calculando aeropuerto: '+aeropuerto+'...')
    dfAeropuerto = pd.DataFrame(
        columns=['Intervalo','PaxPredicted', 'NumDesk', 'Lambda', 'Mu', 'Ro', 'RoSat', 
                 'P0', 'Lq', 'Ls', 'Wq', 'Ws'],
        index=range(24))

    for index, intervalo in dfAeropuertos.iterrows():
        # Número de mostradores
        qtC = 0
        # Condición de Saturación
        qtRoSat = 99
        # Tiempo de espera para servicio
        qtWs = 99
        
        # Calculamos los datos de colas 
        PaxPredicted = int(intervalo[aeropuerto])
        
        # Si el número de pasajeros es 0, guardamos 0 en todos los campos.
        if PaxPredicted == 0:
            dfAeropuerto.iloc[index] = (str(index)+'-'+str(index+1),0, 'NA', 'NA', 'NA', 'NA', 'NA', 
                                        'NA', 'NA', 'NA', 'NA', 'NA')
            continue
            
        qtLambda = PaxPredicted/60
        qtMu = tMServicio
        qtRo = (qtLambda/qtMu)
        
        # Si se cumple la condición de saturación, aumentamos el número de mostradores
        while (qtRoSat >= 1) or (qtWs > tMEspera):
            qtC+=1
            qtRoSat = (qtLambda/(qtC*qtMu))
            qtP0 = (1/(sumatoriaRo(qtC,qtRo)+(((qtRo**qtC)/math.factorial(qtC))*(1/(1-qtRoSat)))))
            qtLq = (qtP0*((qtRo**(qtC+1))/(math.factorial(qtC-1)*((qtC-qtRo)**2))))
            qtLs =((qtLambda*qtMu*(qtRo**qtC)*qtP0)/(math.factorial(qtC-1)*((qtC*qtMu)-qtLambda)**2))+qtRo
            qtWq = qtLq/qtLambda
            qtWs = qtLs/qtLambda
        
        # En este punto tenemos la solución de mostradores
        dfAeropuerto.iloc[index] = (str(index)+'-'+str(index+1), PaxPredicted, qtC, 
                                    qtLambda, qtMu, qtRo, qtRoSat, qtP0, qtLq, qtLs, qtWq, qtWs)
        
    file = FileNameOUT+"CHK_"+aeropuerto+'.xlsx'
    dfAeropuerto.to_excel (file, index = False, header=True)
    print('FIN: Creado Excel ' + file)
        

Calculando aeropuerto: LPA...
FIN: Creado Excel TFM_FileOUT_CHK_LPA.xlsx
Calculando aeropuerto: ACE...
FIN: Creado Excel TFM_FileOUT_CHK_ACE.xlsx
Calculando aeropuerto: FUE...
FIN: Creado Excel TFM_FileOUT_CHK_FUE.xlsx
Calculando aeropuerto: TFN...
FIN: Creado Excel TFM_FileOUT_CHK_TFN.xlsx
Calculando aeropuerto: TFS...
FIN: Creado Excel TFM_FileOUT_CHK_TFS.xlsx
Calculando aeropuerto: SPC...
FIN: Creado Excel TFM_FileOUT_CHK_SPC.xlsx
Calculando aeropuerto: VDE...
FIN: Creado Excel TFM_FileOUT_CHK_VDE.xlsx
Calculando aeropuerto: GMZ...
FIN: Creado Excel TFM_FileOUT_CHK_GMZ.xlsx
